In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', as_supervised=True, with_info=True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image,label

scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)
test_data = scaled_test_data.shuffle(BUFFER_SIZE)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
NUM_EPOCHS = 10

model.fit(train_data, epochs= NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), validation_steps=10, verbose=2)

Epoch 1/10
540/540 - 15s - loss: 0.3417 - accuracy: 0.9017 - val_loss: 0.1792 - val_accuracy: 0.9460
Epoch 2/10
540/540 - 15s - loss: 0.1415 - accuracy: 0.9581 - val_loss: 0.1103 - val_accuracy: 0.9678
Epoch 3/10
540/540 - 15s - loss: 0.1000 - accuracy: 0.9703 - val_loss: 0.0882 - val_accuracy: 0.9758
Epoch 4/10
540/540 - 15s - loss: 0.0753 - accuracy: 0.9774 - val_loss: 0.0850 - val_accuracy: 0.9745
Epoch 5/10
540/540 - 16s - loss: 0.0622 - accuracy: 0.9807 - val_loss: 0.0706 - val_accuracy: 0.9805
Epoch 6/10
540/540 - 15s - loss: 0.0503 - accuracy: 0.9844 - val_loss: 0.0665 - val_accuracy: 0.9800
Epoch 7/10
540/540 - 16s - loss: 0.0414 - accuracy: 0.9870 - val_loss: 0.0464 - val_accuracy: 0.9873
Epoch 8/10
540/540 - 15s - loss: 0.0365 - accuracy: 0.9886 - val_loss: 0.0423 - val_accuracy: 0.9888
Epoch 9/10
540/540 - 16s - loss: 0.0288 - accuracy: 0.9908 - val_loss: 0.0401 - val_accuracy: 0.9882
Epoch 10/10
540/540 - 15s - loss: 0.0261 - accuracy: 0.9917 - val_loss: 0.0298 - val_accura

In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 3s 3s/step - loss: 0.0731 - accuracy: 0.97 - 3s 3s/step - loss: 0.0731 - accuracy: 0.9782

In [8]:
test_accuracy

0.9782

In [9]:
test_loss

0.07310646027326584